In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Load the CSV file containing the User IDs and URLs
df_csv = pd.read_csv('User_Review_url_and_ID.csv')

# Headers to simulate a browser
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "max-age=0",
    "cookie": 'ccsid=997-0059822-9974992; locale=en; blocking_sign_in_interstitial=true; csm-sid=448-9044706-8238887; session-id=138-9449735-4650719; lc-main=en_US; ubid-main=132-0831767-7645715; csm-hit=tb:s-HH6TEBC541XWDPX2HW08|1728400665806&t:1728400665807; session-id-time=2359120669l; session-token="9epd7M6HsVRPHteGVvJu5Fk8d+KvASerta8tz+la5w+TVREtncxVnfcCzs8K9EpSg+fQrgx1GgnsD2rLlayR4DmjBe1MsI/hzqbpLxW9fzltLac/Bx4GP/iysuf1f336QEHPYkFM5wvwKAJRlJF/gUtvoqUNgeHO7d0nUgwzQ4n10cVaWcJP8kU3S8U/EG5f1z4kuWhJEC/GxOGAFPM2AwR1aAfDTGc3oUmUkI4heyVFL2PVr5DhINiO23tre2UqOAtOSgArJ6foDhvdkRwB1A2cwygZeAAPu6znpJ+cxdwCK+Qw0zKdF5L4dt5MPkikWWxVqpvSWD4krg+mGhL2EJhFdAQbaWn2SM7qiWrwNN8="; x-main="qY09NsWCzSNW8b933Yonj4af6qs5TDB0DUsYapgS18@nyKFkHdJj5p@Mt2dldE6F"; at-main=Atza|IwEBIKKe6dZi9mkGtv2CtS3qsv9NuaJNSNpmNpDSxRXvt17a_x2-zjEpiAOHkEQ34JuOVngoGfAl8dIqliYqOigT30w9CGWKtZI6rgDWz0mruXs-yNgr_nsxBWgaXid-1e8ZvmgQryKo5OgAIx-Lce0ocNWWSfh3g77KeZNETFdonWPAuJz5V82UuuKblmr-o03ZzeyBC7rbWUbFLRhtcSWy7Mqogs8ONE5dM3XgdBITj5b5F2h4Z_ih2WgCM4wTps3r6HY; sess-at-main="l6swvzfG2thASaG98K8Nu/j0D/+teOfY5a1UhpsZBz8="; likely_has_account=true; *session*id2=4cd4cabfc8c16e891c0f62348aa076dd',
    "if-none-match": 'W/"862d60fdc6915eed33d69a373690f36d"',
    "priority": "u=0, i",
    "sec-ch-ua": '"Google Chrome";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "none",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36"
}

# Initialize a DataFrame to store ratings
ratings_data = []

# Iterate through each User_ID in the CSV file
for index, row in df_csv.iterrows():
    user_id = row['User_ID']
   # print(f"Processing User ID: {user_id}")
    
    for i in range(1, 2):# Loop through pages if needed, modify the range accordingly
        Book_id = []
        Rating = []
        unique_book = []
       # print(f"Scraping page {i}")
        url = f"https://www.goodreads.com/review/list/{user_id}?order=d&page={i}&sort=rating&view=reviews"
       # print(f"Scraping URL: {url}")
        
        # Request the page
        response = requests.get(url, headers=headers)

        # Parse the content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        rating_tds = soup.find_all('td', class_='field rating')
        # Find book URLs and IDs from 'div' with class 'value'
        tempa = soup.find_all('div', class_='value')
        
        for div in tempa:
            a_tag = div.find('a', href=True)
            if a_tag:
                book_url = a_tag['href']
                if '/book/show/' in book_url:
                    # Extract book_id before '-' and '.'
                    book_id = book_url.split('/book/show/')[1].split('-')[0]
                    book_id = book_id.split('.')[0]
                    # print(book_id)
                    Book_id.append(book_id)
        for item in Book_id:
            if item not in unique_book:
                unique_book.append(item)
                #print(item)
                    
# Debug: Check if any 'field rating' td elements are found
        if not rating_tds:
            print("No 'field rating' td elements found. Please verify the class name and HTML structure.")
        else:
            # Iterate over each 'field rating' td element
            for rating_td in rating_tds:
                # Find all `span` elements inside with the class 'staticStar p10'
                stars = rating_td.find_all('span', class_='staticStar p10')
                # Append the count of stars to the ratings list
                # ratings.append(len(stars))
                # print("ratting ")
                # print(len(stars))
                Rating.append(len(stars))

        i=0
        while i < len(unique_book):
        # Access the element at index i
            #print('hello')
            # print(unique_book[i])
            #print('rating')
            ratings_data.append({
                'User_ID': user_id,
                'Book_ID': unique_book[i],
                'Rating': Rating[i]
            })

           # print(Rating[i])
            # Increment the index
            i += 1                

# Convert to DataFrame
ratings_df = pd.DataFrame(ratings_data)

# Display or save the DataFrame
print(ratings_df)

# Optionally, save to a CSV file
ratings_df.to_csv('scraped_ratings.csv', index=False)


Processing User ID: 14008088
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
14116
14118
24812
72915
72910
76234
77174
7356990
87420
140130
146122
146128
166353
166356
166396
166399
179173
9602495
22446777
13498737
Processing User ID: 107704225
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
ratting 
5
28575405
36381681
9602495
13498737
26170109
583569
8998592
24523250
34835814
55310550
12664136
31118343
5784403
920607
36634145
16123795
12338591
24479313
11386578
17828716
17236862


IndexError: list index out of range